In [ ]:
import pandas as pd
import csv
import re

import nltk
nltk.download('punkt')
nltk.download("stopwords")

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
!pip install pymorphy2
!pip install pymystem3
# !pip install git+https://github.com/nlpub/pymystem3
# !pip install -q http://download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz
from pymystem3 import Mystem
import pymorphy2
from string import punctuation

!pip install pandarallel
from pandarallel import pandarallel

from google.colab import drive
drive.mount('/content/drive')


%cd /content/drive/My\ Drive/SNA2020/Project

df_bots=pd.read_csv('./bots_update.tsv',sep="\t",header=0)
df_bots.head()

df_politics = pd.read_csv('./politics_update.tsv',sep="\t",header=0)
df_bots.head()

url_stopwords_ru = "https://raw.githubusercontent.com/stopwords-iso/stopwords-ru/master/stopwords-ru.txt"

badwords = [
  u'я', u'а', u'да', u'но', u'тебе', u'мне', u'ты', u'и', u'у', u'на', u'ща', u'ага',
  u'так', u'там', u'какие', u'который', u'какая', u'туда', u'давай', u'короче', u'кажется', u'вообще',
  u'ну', u'не', u'чет', u'неа', u'свои', u'наше', u'хотя', u'такое', u'например', u'кароч', u'как-то',
  u'нам', u'хм', u'всем', u'нет', u'да', u'оно', u'своем', u'про', u'вы', u'м', u'тд',
  u'вся', u'кто-то', u'что-то', u'вам', u'это', u'эта', u'эти', u'этот', u'прям', u'либо', u'как', u'мы',
  u'просто', u'блин', u'очень', u'самые', u'твоем', u'ваша', u'кстати', u'вроде', u'типа', u'пока', u'ок'
]

emoticons = [u'&#_128514;', u'&#_128517;', u'u&#34;\U0001F923&#34;']

emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U0001F923"
        u"\U0001F9A0"
        u"\U0001F000-\U0001FFFF"
        u"\U00002639"
                           "]+", flags=re.UNICODE)

def get_text(url, encoding='utf-8', to_lower=True):
    url = str(url)
    if url.startswith('http'):
        r = requests.get(url)
        if not r.ok:
            r.raise_for_status()
        return r.text.lower() if to_lower else r.text
    elif os.path.exists(url):
        with open(url, encoding=encoding) as f:
            return f.read().lower() if to_lower else f.read()
    else:
        raise Exception('parameter [url] can be either URL or a filename')

def preprocess_text_mention_and_n(text):
  if text.find(" ") - 1 > 0 and text[text.find(" ") - 1] == ',':
    text = text[text.find(" ") + 1:]
  text = text.replace('\\n', ' ')
  return text

def clean_emoticons(text):
  for word in emoticons:
    text = text.replace(word, '')
  text = emoji_pattern.sub(r'', text)
  return text

def preprocess_text_lemmatize(text, setting="pymorphy"):
    text = text.lower()
    if setting == "mystem":
      mystem = Mystem()
      tokens = mystem.lemmatize(text)
    elif setting == "pymorphy":
      tokens = word_tokenize(text, language="russian")
      morph = pymorphy2.MorphAnalyzer()
      tokens = [morph.parse(token)[0].normal_form for token in tokens]
    else:
       raise Exception('parameter setting should be fill')
    tokens = [token for token in tokens if token.strip() not in punctuation]
    tokens = " ".join(tokens)
    tokens = tokens.replace("``", '').replace("''", '').replace(".", '').replace("«", '').replace("»", '').replace("—", '').replace("№", '')
    for symbol in punctuation:
      tokens = tokens.replace(symbol, '')
    for symbol in ['1', '2', '3', '4', '5', '6', '7', '8', '9', '0']:
      tokens = tokens.replace(symbol, '')
    # print(tokens)
    return tokens

def preprocess_text_stopwords(tokens, setting):
    if setting == "nltk-stopwords":
      stopwords = stopwords.words("russian")
    elif setting == "github-stopwords":
      stopwords = get_text(url_stopwords_ru).splitlines()
    elif setting == "badwords":
      stopwords = badwords
    else:
       raise Exception('parameter setting should be fill')
    
    tokens = [token for token in tokens if token not in stopwords \
              and len(token) > 2 \
              and token != " " \
              and token.strip() not in punctuation]
    return tokens

pandarallel.initialize()

# df_bots['text']=df_bots['text'].parallel_apply(preprocess_text_mention_and_n)
# df_bots['text']=df_bots['text'].parallel_apply(clean_emoticons)
# df_bots['pymorphy']=df_bots['text'].parallel_apply(preprocess_text_lemmatize)
# print(df_bots['pymorphy'][100:200])
# df_bots.to_csv('bots_pymorphy.tsv',sep='\t', quoting=csv.QUOTE_NONE)

df_politics['text']=df_politics['text'].parallel_apply(preprocess_text_mention_and_n)
df_politics['text']=df_politics['text'].parallel_apply(clean_emoticons)
df_politics['pymorphy']=df_politics['text'].parallel_apply(preprocess_text_lemmatize)
print(df_politics['pymorphy'][100:200])
df_politics.to_csv('politics_pymorphy.tsv',sep='\t', quoting=csv.QUOTE_NONE)

